# MPS canonical forms 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

import numpy as np

import copy 

import myMPSstuff as mps
import myMPOstuff as mpo

import isingMPO as isi
import applMPOMPS as mpomps

from myUtils import sncon as ncon
from myUtils import real_close as rc
from myUtils import checkIdMatrix

import myEnvironmentsMPO as envs



In [3]:

test2 = mps.myMPS(mps.randMPS(chi=100))
print(f"Entropies=  {test2.getEntropies()}")

print(type(test2))
print(type(test2.MPS))


3 sweeps
Entropies=  [0.0, 2.694595076367125e-05, 6.327220788930835e-07, 4.7570257225485857e-07, 3.505242920488149e-07, 3.020487153824784e-07, 2.786561285450707e-07, 3.5912698970549453e-07, 2.933263188532166e-07, 0.00023936977941098396, 0.0]
<class 'myMPSstuff.myMPS'>
<class 'list'>


In [4]:

testMPO = mpo.myMPO(isi.IsingMPO(LL=10, J = 1, g = 0.4))


In [5]:

#mpstest = np.random.rand(4,7,3,7)

test = mps.myMPS()

print(test.chis)
print([np.shape(s) for s in test.MPS])

print(test.getNormSlow())
print(test.getNorm())



[1, 20, 20, 20, 20, 20, 20, 1]
[(1, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 1, 2)]
(3818579710500857.5-0.0625j)
61794657.62103435


In [6]:
test = mps.myMPS(mps.plusState(10))

print(test.chis)
print([np.shape(s) for s in test.MPS])

# the state should be normalized already 
print(test.getNormSlow())
print(test.getNorm())

test.bringCan()
print(test.getEntropies())  # should be zero for a prod state



[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[(1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2)]
0.9999999999999978
0.9999999999999992
3 sweeps
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [7]:
test = mps.myMPS()
print(test.getNormSlow())

print(test.getNorm())
test.bringCan()

(3818579710500857.5-0.0625j)
61794657.62103435
3 sweeps


'L'

In [8]:


test3 = mps.myMPS(mps.randMPS(LL=20, chi=200))
print(f"Entropies=  {test3.getEntropies()}")

testMPO = mpo.myMPO()




3 sweeps
Entropies=  [0.0, 1.2573890686017716e-05, 1.999302538565336e-07, 8.069839711398202e-08, 1.0426632435772923e-08, 8.259309608427732e-08, 1.0381344172403728e-07, 4.8715736842272496e-08, 6.530039362639734e-09, 1.4586232945557017e-08, 2.233489557158525e-08, 2.6121404313700047e-08, 7.304817147256808e-09, 1.75820659581536e-07, 1.8981308427669686e-07, 1.6995870780581854e-07, 4.697375539224514e-09, 5.5587312041864756e-08, 7.584879139772995e-08, 2.0056930684263962e-05, -1.3322676295501888e-15]


## Check that we're actually in proper L-R canon forms by contracting a bit and see if we recover identity matrices and normalizations 

In [9]:
mymps3 = mps.myMPS(mps.randMPS(40,chi=300))

mymps3.bringCan(epsTrunc=1e-10,epsNorm=1e-14, chiMax=200)
print(mymps3.curr_form)
# print(mymps3.SV)
# print(mymps3.SVinv)
print(mymps3.chis)

# Now set to R 
mymps3.set_form(mode='R')
print(mymps3.curr_form)
# So now the MPS should be normalized and in  RIGHT canonical form 
# Check by contracting some B's 

print(f"Norm={mymps3.getNorm()}")

[ checkIdMatrix(ncon([m,np.conj(m)],[[-1,1,2],[-2,1,2]])) for m in mymps3.MPS ]


mymps3.set_form(mode='L')
print(mymps3.curr_form)
# print(mymps3.SV)
# print(mymps3.SVinv)


# So now the MPS should be normalized and in  LEFT canonical form 
# Check by contracting some A's 

print(f"Norm={mymps3.getNorm()}")

[ checkIdMatrix(ncon([m,np.conj(m)],[[1,-1,2],[1,-2,2]])) for m in mymps3.MPS ]

# # Now set to R again 
# mymps3.set_form(mode='R')
# print(mymps3.curr_form)
# # So now the MPS should be normalized and in  RIGHT canonical form 
# # Check by contracting some B's 

# print(f"Norm={mymps3.getNorm()}")

# [ checkIdMatrix(ncon([m,np.conj(m)],[[-1,1,2],[-2,1,2]])) for m in mymps3.MPS ]



3 sweeps
L
[1, 2, 4, 7, 5, 4, 5, 4, 5, 5, 4, 4, 5, 4, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 7, 4, 2, 1]
R
Norm=1.0000000000000022
identity, size = 1
identity, size = 2
identity, size = 4
identity, size = 7
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 7
identity, size = 4
identity, size = 2
L
Norm=1.0000000000000018
identity, size = 2
identity, size = 4
identity, size 

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [10]:
m =mymps3.MPS[8]
np.real_if_close(np.around(ncon([m,np.conj(m)],[[1,-1,2],[1,-2,2]]),12))

print(mymps3.SV[7][0])
print(np.linalg.norm((mymps3.SV[7])))
print(mymps3.SV[8])
print(np.linalg.norm((mymps3.SV[8])))
print(mymps3.SVinv[8])
print(np.linalg.norm((mymps3.SVinv[8])))

0.9999999989584117
1.0000000000000004
[1.00000000e+00 2.84719379e-05 6.82247474e-08 4.59255367e-09
 1.66368874e-10]
1.0000000000000004
[1.00000000e+00 3.51223019e+04 1.46574379e+07 2.17743781e+08
 6.01073973e+09]
6014700267.230909


## Try computing an expectation value, first the "hard" way (ncon the whole thing), then using the right-canonical expression

In [11]:
# Try to do it a bit better, define separately psi and psi*
# and compute the norm 

testL = 12
testd = 2
testChi = 8

mps1 = mps.randMPS(testL,chi=testChi, d=testd)

mypsi = mps.myMPS(mps1)
#mypsic = mps.myMPS([np.conj(m) for m in mps1])

# make two copies for later messing around 
mypsic1 = copy.deepcopy(mypsi)
mypsic1conj = mps.myMPS([np.conj(m) for m in mypsic1.MPS])

mypsic2 = copy.deepcopy(mypsi)


# Now define the operator to exp-value,
# say we wanna compute <sigma_x> 
op1 = np.array([[0,1.],[1.,0]])

# at site r = 3 

print("Exp value the hard way: ")

insertjj = 3 
tryind = [[1,3,2]]
tryind.append([1,4,2])

for jj in range(1,testL):
    tj = 3*jj
    if jj == insertjj: # horrible hack 
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+2,9999])
        tryind.append([tj+1,tj+4,9999])
    else:
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+1,tj+4,tj+2])

tryind[-2][1] = tryind[-1][1]

opList = []
for jj, m in enumerate(mypsi.MPS):
    if jj == insertjj:
        opList.append(m)
        opList.append(op1)
        opList.append(m.conj())
    else:
        opList.append(m)
        opList.append(m.conj())

ev = ncon(opList,tryind)

#print(tryind )
print(np.real_if_close(ev/mypsi.getNormSlow() ))
#print(np.real_if_close(ev/mypsi.getNorm() ))

print("Second way: apply the op to the relevant site, then compute the overlap with itself")
mypsic1.MPS[3] = ncon([op1,mypsic1.MPS[3]],[[-3,1],[-1,-2,1]])
expvaloverlap = mypsic1.overlap(mypsic1conj) 
print(np.real_if_close(expvaloverlap/mypsic1.getNormSlow() ))

print(" Now compute it with the canonical form ")

mypsic2.set_form('R')

conTen = [np.diag(mypsic2.SV[3]),np.diag(mypsic2.SV[3]),mypsic2.MPS[3],np.conj(mypsic2.MPS[3]),op1]
conIdx = [[1,2],[1,3],[3,5,4],[2,5,6],[4,6]]

[print(np.shape(elem)) for elem in conTen]
print(len(conTen),len(conIdx))

print(np.real_if_close(ncon(conTen,conIdx)))


Exp value the hard way: 
0.983674957343944
Second way: apply the op to the relevant site, then compute the overlap with itself
0.9836749573439441
 Now compute it with the canonical form 
3 sweeps
(8, 8)
(8, 8)
(8, 8, 2)
(8, 8, 2)
(2, 2)
5 5
0.983674957343945


# build and test environments

In [12]:
o = mpo.myMPO(isi.IsingMPO(LL = 20))

psi = mps.myMPS(mps.randMPS(LL=o.LL))

In [13]:
le, re = envs.build_environments(psi, o)

print(mpomps.expValMPO(psi, o))

for j in range(0,psi.LL):
    #print(np.shape(le[j]), np.shape(re[j]), np.shape(re[j+1]))
    expval = np.real_if_close(ncon([le[j],re[j]],[[1,2,3],[1,2,3]]))
    print(j, expval)


3 sweeps
-7.541504759916814
0 -7.541504759916804
1 -7.541504759916805
2 -7.541504759916807
3 -7.541504759916806
4 -7.541504759916805
5 -7.541504759916805
6 -7.541504759916804
7 -7.541504759916806
8 -7.541504759916805
9 -7.541504759916807
10 -7.541504759916806
11 -7.541504759916807
12 -7.541504759916808
13 -7.5415047599168075
14 -7.5415047599168075
15 -7.541504759916808
16 -7.541504759916808
17 -7.541504759916807
18 -7.541504759916807
19 -7.541504759916807


In [14]:
print(rc(le[-1])) # should be a number

[[[-7.54150476]]]
